In [32]:
import pandas as pd
import plotly.express as px

In [33]:
dados = pd.read_csv('../dados_modelo.csv')
df = pd.DataFrame(dados)

In [34]:
df.head().round(2)

,Ordem de prod,Ref. do Artigo,Duração da ordem,Temperatura (F°),Peso médio (g),Comprimento médio (mm),Diâmetro (mm),Gotas cortadas (toneladas),Total de Frascos,Total de Defeitos,Sujo de óleo,Aderido,Dobra,Trinca por choque térmico,Ombro mal cheio,Racho na terminação,Lascado em produção,Frio
0,192970,C0516.0000R,9,2125.51,169.91,125.51,30.97,178.79,1052031,2148,437,2,77,176,68,28,12,7
1,193112,C1164.0000R,3,2173.68,168.17,88.73,38.57,44.73,265727,584,259,0,35,22,0,9,2,0
2,193164,C1280.0000R,12,2119.41,147.47,100.12,35.43,236.13,1600487,2661,851,2,43,129,0,220,33,19
3,193206,C1284.0000R,4,2170.66,171.15,90.40,36.88,74.31,433842,1041,245,0,26,6,0,65,10,0
4,193207,C1298.0000R,10,2139.29,170.87,98.23,36.56,136.92,801014,2130,754,0,27,87,0,79,45,4


### Trabalhando com defeitos por dia 

In [35]:
dados_dias = pd.read_csv('../dados_dias_modelo.csv')
df_dias = pd.DataFrame(dados_dias)

In [36]:
df_dias.head()

,Ordem de prod,Dia Contado,Dias de Duração,Dia Normalizado,Total de Defeitos no Dia
0,192970,1,9,0.111111,222
1,192970,2,9,0.222222,249
2,192970,3,9,0.333333,268
3,192970,4,9,0.444444,242
4,192970,5,9,0.555556,232


### Visualização

Duração por complexidade

In [37]:
fig1 = px.scatter(df, 
                  x='Duração da ordem', 
                  y='Temperatura (F°)', 
                  title='Duração da ordem vs Temperatura (F°)', 
                  color='Temperatura (F°)',  # Cor de acordo com a Temperatura
                  size='Peso médio (g)',    # Tamanho de acordo com o Peso médio
                  color_continuous_scale='peach')  # Definindo uma escala de cores
fig1.show()

# Plotando Duração da ordem vs Peso médio com cor e tamanho
fig2 = px.scatter(df, 
                  x='Duração da ordem', 
                  y='Peso médio (g)', 
                  title='Duração da ordem vs Peso médio (g)', 
                  color='Peso médio (g)',  # Cor de acordo com o Peso médio
                  size='Temperatura (F°)',  # Tamanho de acordo com a Temperatura
                  color_continuous_scale='peach')  # Escala de cores personalizada
fig2.show()

# Plotando Duração da ordem vs Comprimento médio com cor e tamanho
fig3 = px.scatter(df, 
                  x='Duração da ordem', 
                  y='Comprimento médio (mm)', 
                  title='Duração da ordem vs Comprimento médio (mm)', 
                  color='Comprimento médio (mm)',  # Cor de acordo com o Comprimento médio
                  size='Peso médio (g)',    # Tamanho de acordo com o Peso médio
                  color_continuous_scale='peach')  # Escala de cores personalizada
fig3.show()

# Plotando Duração da ordem vs Diâmetro com cor e tamanho
fig4 = px.scatter(df, 
                  x='Duração da ordem', 
                  y='Diâmetro (mm)', 
                  title='Duração da ordem vs Diâmetro', 
                  color='Diâmetro (mm)',  # Cor de acordo com o Diâmetro
                  size='Diâmetro (mm)',  # Tamanho de acordo com a Temperatura
                  color_continuous_scale='peach')  # Escala de cores personalizada
fig4.show()

fig5 = px.scatter(df, 
                  x='Duração da ordem', 
                  y='Total de Frascos', 
                  title='Duração da ordem vs Total de Frascos', 
                  color='Total de Frascos',  # Cor de acordo com o Diâmetro
                  size='Total de Frascos',  # Tamanho de acordo com a Temperatura
                  color_continuous_scale='peach')  # Escala de cores personalizada
fig5.show()

> Os gráficos indicam baixa variância nos dados e/ou ausência de um relacionamento claro entre as variáveis.

> Duração da ordem não é influenciada pela 'complexidade' do frasco.

Defeitos por dia contado

In [38]:
defeitos_por_dia_contado = df_dias.groupby('Dia Contado')[['Total de Defeitos no Dia']].mean().reset_index()
defeitos_por_dia_contado.columns = ['Dia Contado', 'Média de Defeitos']

In [39]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=defeitos_por_dia_contado['Dia Contado'],
    y=defeitos_por_dia_contado['Média de Defeitos'],
    mode='lines+markers', 
    marker=dict(size=10),
    line=dict(color='red'),
    name='Média de Defeitos'
))

fig.update_layout(
    title='Média de Defeitos por Dia',
    title_font_size=16,
    xaxis=dict(
        title='Dia',
        title_font_size=14,
        dtick=1 
    ),
    yaxis=dict(
        title='Média de Defeitos',
        title_font_size=14
    )
)

fig.show()


## Implementando Modelo

### Definindo variáveis e teste

In [40]:
from sklearn.model_selection import train_test_split

# Selecionando as variáveis de entrada e saída
X = df[['Peso médio (g)', 'Comprimento médio (mm)', 
        'Diâmetro (mm)', 'Total de Frascos']]

y = df[['Duração da ordem', 'Total de Defeitos']]

# Divisão em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import math

# Instanciando o modelo
model = RandomForestRegressor(random_state=42, n_estimators=100)

# Treinando o modelo
model.fit(X_train, y_train)

# Fazendo previsões
y_pred = model.predict(X_test)

# Separando previsões para 'Duração da ordem' e 'Total de Defeitos'
y_test_duracao = y_test['Duração da ordem']
y_test_defeitos = y_test['Total de Defeitos']
y_pred_duracao = y_pred[:, 0]
y_pred_defeitos = y_pred[:, 1]

# Avaliação do modelo
print(f"Mean Squared Error (Duração da ordem): {mean_squared_error(y_test_duracao, y_pred_duracao):.2f}")
print(f"R² Score (Duração da ordem): {r2_score(y_test_duracao, y_pred_duracao):.2f}")
print(f"Mean Squared Error (Total de Defeitos): {mean_squared_error(y_test_defeitos, y_pred_defeitos):.2f}")
print(f"R² Score (Total de Defeitos): {r2_score(y_test_defeitos, y_pred_defeitos):.2f}")

Mean Squared Error (Duração da ordem): 0.56
R² Score (Duração da ordem): 0.77
Mean Squared Error (Total de Defeitos): 28057.65
R² Score (Total de Defeitos): 0.61


### Testando Modelo

In [42]:
# Dados de uma nova ordem
nova_ordem = [[180.5, 50.5, 12.5, 700000]] 
#'Peso médio (g)', 'Comprimento médio (mm)', 'Diâmetro (mm)', 'Total de Frascos'
nova_ordem_nome_colunas = ['Peso médio (g)', 'Comprimento médio (mm)', 'Diâmetro (mm)', 'Total de Frascos']

nova_predicao = model.predict(nova_ordem)

nova_duracao_prevista = math.ceil(nova_predicao[0][0])  
novo_total_defeitos_previsto = nova_predicao[0][1]  

print(f"Duração prevista da ordem: {nova_duracao_prevista} dias")
print(f"Total de defeitos previstos: {novo_total_defeitos_previsto:.2f}")

# Adicionando mais informações sobre a previsão
print("\nDetalhes da previsão:")
for i, coluna in enumerate(nova_ordem_nome_colunas):
    print(f"{coluna}: {nova_ordem[0][i]}")
# Fazendo a previsão
nova_predicao = model.predict(nova_ordem)

# Extraindo as previsões
nova_duracao_prevista = math.ceil(nova_predicao[0][0])  # Previsão da duração
novo_total_defeitos_previsto = nova_predicao[0][1]  # Previsão do total de defeitos

print(f"Duração prevista da ordem: {nova_duracao_prevista} dias")
print(f"Total de defeitos previstos: {novo_total_defeitos_previsto:.2f}")

Duração prevista da ordem: 8 dias
Total de defeitos previstos: 1663.03

Detalhes da previsão:
Peso médio (g): 180.5
Comprimento médio (mm): 50.5
Diâmetro (mm): 12.5
Total de Frascos: 700000
Duração prevista da ordem: 8 dias
Total de defeitos previstos: 1663.03


c:\Users\evosystem02.ti\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

c:\Users\evosystem02.ti\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names



**`math.ceil()`**:
   - Garante que `nova_duracao_prevista` seja arredondada sempre para cima.
   - Por exemplo: `2.50` será arredondado para `3`, e `3.01` também será arredondado para `4`.


> Arredondar para o próximo número inteiro (em vez de truncar) torna o cálculo mais conservador, especialmente em cenários onde o tempo de duração deve ser superestimado para evitar problemas.

### Distribuição dos defeitos

Distribuindo os defeitos previstos proporcionalmente com base na média histórica:

In [43]:
# Soma total das médias de defeitos
total_medio_defeitos = defeitos_por_dia_contado['Média de Defeitos'].sum()

# Calcula os defeitos previstos proporcionalmente
defeitos_por_dia_contado['Defeitos Previstos'] = (
    (defeitos_por_dia_contado['Média de Defeitos'] / total_medio_defeitos) * novo_total_defeitos_previsto
)

Garantimos que apenas os dias que correspondem à duração prevista da nova ordem sejam considerados:

In [44]:
# Seleciona apenas os dias da nova ordem
df_dias_duracao = defeitos_por_dia_contado.head(nova_duracao_prevista).copy()

Certificando que a soma total dos defeitos para os dias selecionados seja igual ao total de defeitos esperado

In [45]:
# Soma dos defeitos previstos para os dias selecionados
soma_defeitos_dia = df_dias_duracao['Defeitos Previstos'].sum()

# Calcula o fator de ajuste
ajuste_fator = novo_total_defeitos_previsto / soma_defeitos_dia

# Aplica o ajuste nos defeitos previstos
df_dias_duracao.loc[:, 'Defeitos Previstos'] *= ajuste_fator

Visualização

In [46]:
# Gráfico de previsão de defeitos
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_dias_duracao['Dia Contado'],
    y=df_dias_duracao['Defeitos Previstos'],
    mode='lines+markers',
    marker=dict(size=10),
    line=dict(color='blue'),
    name='Defeitos Previstos'
))

fig.update_layout(
    title=f'Previsão de Defeitos ao Longo dos {nova_duracao_prevista} Dias da Nova Ordem',
    title_font_size=16,
    xaxis=dict(
        title='Dia de Produção',
        title_font_size=14,
        dtick=1  # Alterar para 2, 5, etc., conforme necessário
    ),
    yaxis=dict(
        title='Defeitos Previstos',
        title_font_size=14
    )
)

print(f"Duração prevista da ordem: {nova_duracao_prevista} dias")
print(f"Total de defeitos previstos: {novo_total_defeitos_previsto:.2f}")

fig.show()

df_dias_duracao[['Dia Contado', 'Defeitos Previstos']].set_index('Dia Contado').round(2)

Duração prevista da ordem: 8 dias
Total de defeitos previstos: 1663.03


,Defeitos Previstos
Dia Contado,
1,215.60
2,237.13
3,207.63
4,213.25
5,207.98
6,206.29
7,181.13
8,194.02


### Como seria a soma com o total solicitado: 

O total produzido esperado é a soma do número de frascos solicitados pelo cliente e os defeitos previstos.

In [47]:
# Calculando o total produzido esperado
total_frascos_solicitados = nova_ordem[0][3]  # Total solicitado (ex.: 123456)
total_produzido_esperado = total_frascos_solicitados + math.ceil(novo_total_defeitos_previsto)

In [48]:
print(f"Total solicitados: {total_frascos_solicitados} frascos")
print(f"Total produzido esperado: {total_produzido_esperado} frascos em {nova_duracao_prevista} dias")

Total solicitados: 700000 frascos
Total produzido esperado: 701664 frascos em 8 dias


In [49]:
# Gráfico de previsão de defeitos
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_dias_duracao['Dia Contado'],
    y=df_dias_duracao['Defeitos Previstos'],
    mode='lines+markers',
    marker=dict(size=10),
    line=dict(color='blue'),
    name='Defeitos Previstos'
))

fig.update_layout(
    title=f'Previsão de Defeitos ao Longo dos {nova_duracao_prevista} Dias da Nova Ordem',
    title_font_size=16,
    xaxis=dict(
        title='Dia de Produção',
        title_font_size=14,
        dtick=1  # Alterar para 2, 5, etc., conforme necessário
    ),
    yaxis=dict(
        title='Defeitos Previstos',
        title_font_size=14
    )
)

print(f"Total solicitado: {total_frascos_solicitados} frascos")
print(f"Duração prevista da ordem: {nova_duracao_prevista} dias")
print(f"Total de defeitos previstos: {novo_total_defeitos_previsto:.2f}")
print(f"Total produzido esperado: {total_produzido_esperado} frascos em {nova_duracao_prevista} dias")

fig.show()

df_dias_duracao[['Dia Contado', 'Defeitos Previstos']].set_index('Dia Contado').round(2)

Total solicitado: 700000 frascos
Duração prevista da ordem: 8 dias
Total de defeitos previstos: 1663.03
Total produzido esperado: 701664 frascos em 8 dias


,Defeitos Previstos
Dia Contado,
1,215.60
2,237.13
3,207.63
4,213.25
5,207.98
6,206.29
7,181.13
8,194.02
